# Projecting Concert Ticket Sales & F&B Sales

**Data Sources:**

- Prop TM historical data
- All visible future TM data (sphr_visible_event_data_vf.xlsx)

In [49]:
# imports
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [50]:
tm_data = pd.read_excel('data/sphr_visible_event_data_vf.xlsx')

tm_data['Date'] = pd.to_datetime(tm_data['Date & Time'])
tm_data['Date'] = tm_data['Date'].dt.date

# Drop rows where Group Name is Postcards
tm_data = tm_data[tm_data['Group Name'] != 'Postcards']

groups = tm_data['Group Name'].value_counts()
print(f'Concerts: \n{groups}')

Concerts: 
U2 Residence    30
Dead&Co         18
Phish            4
Name: Group Name, dtype: int64


In [51]:
# make a list that stores the date range from min to max date that appears in the dataframe
date_range = pd.date_range(start=tm_data['Date'].min(), end=tm_data['Date'].max())

# make a dict that stores date: number of shows for each unique group
group_dict = {}
for group in tm_data['Group Name'].unique():
    group_dict[group] = tm_data[tm_data['Group Name'] == group]['Date'].value_counts().reindex(date_range).fillna(0)

# Plot the data as a bar chart
fig = go.Figure()
for group in group_dict:
    fig.add_trace(go.Bar(x=date_range, y=group_dict[group], name=group))

fig.update_layout(
    title='Show Days by Artist',
    xaxis_title='Date',
    yaxis_title='Number of Shows',
    barmode='stack'
)

fig.show()

# Projecting Revenue by Event and Group

In [52]:
# get proprietary hist data
hist_stats = pd.read_excel('prop_data/sphr_hist_stats_data.xlsx')

In [53]:
tm_data.head()

,Group Name,Event Name,Date & Time,Average GA Ticket Price,Total Ticket Units Available,Aggregate Ticket Value,Date
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28T04:00:00Z,195,20000,3900000,2024-01-28
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18T04:00:00Z,195,20000,3900000,2024-02-18
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27T04:00:00Z,195,20000,3900000,2024-01-27
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04T04:00:00Z,195,20000,3900000,2024-02-04
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03T04:00:00Z,195,20000,3900000,2024-03-03


In [54]:
# Init Revenue Dataframe

revenue_df = pd.DataFrame()
revenue_df['Group Name'] = tm_data['Group Name']
revenue_df['Event Name'] = tm_data['Event Name']
revenue_df['Date'] = tm_data['Date']
revenue_df['Total Available Tickets'] = tm_data['Total Ticket Units Available']
revenue_df['Projected Attendance (Bear)'] = 0
revenue_df['Projected Attendance % (Bear)'] = 0
revenue_df['Projected Attendance (Base)'] = 0
revenue_df['Projected Attendance % (Base)'] = 0
revenue_df['Projected Attendance (Bull)'] = 0
revenue_df['Projected Attendance % (Bull)'] = 0
revenue_df['Projected Ticket Revenue (Bear)'] = 0
revenue_df['Projected Ticket Revenue (Base)'] = 0
revenue_df['Projected Ticket Revenue (Bull)'] = 0
revenue_df['Projected F&B Revenue (Bear)'] = 0
revenue_df['Projected F&B Revenue (Base)'] = 0
revenue_df['Projected F&B Revenue (Bull)'] = 0
revenue_df['Projected Merch Revenue (Bear)'] = 0
revenue_df['Projected Merch Revenue (Base)'] = 0
revenue_df['Projected Merch Revenue (Bull)'] = 0

revenue_df.head()

,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


# U2

In [55]:
# mean and std attendance for U2 available in hist_stats
u2_mean = hist_stats['U2 Attendance'].mean()
u2_std = hist_stats['U2 Attendance'].std()
print(f'U2 Mean: {u2_mean}, U2 Std: {u2_std}')

U2 Mean: 0.9023333333333333, U2 Std: 0.05351245960833171


In [56]:
# Bear case is negative two sigma from mean
u2_bear_pct = u2_mean - (2 * u2_std)
print(f'U2 Bear: {u2_bear_pct}')

# Base case is mean
u2_base_pct = u2_mean
print(f'U2 Base: {u2_base_pct}')

# Bull case is positive two sigma from mean (floor at 100%)
u2_bull_pct = min(u2_mean + (2 * u2_std), 1)
print(f'U2 Bull: {u2_bull_pct}')

for index, row in revenue_df.iterrows():
    if row['Group Name'] == 'U2 Residence':
        revenue_df.at[index, 'Projected Attendance (Bear)'] = u2_bear_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bear)'] = u2_bear_pct
        revenue_df.at[index, 'Projected Attendance (Base)'] = u2_base_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Base)'] = u2_base_pct
        revenue_df.at[index, 'Projected Attendance (Bull)'] = u2_bull_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bull)'] = u2_bull_pct

revenue_df.head()

U2 Bear: 0.7953084141166699
U2 Base: 0.9023333333333333
U2 Bull: 1


,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0


# Phish (sold out)

In [57]:
# mean and std attendance for Phish available in hist_stats
phish_mean = hist_stats['Phish Attendance'].mean()
phish_std = hist_stats['Phish Attendance'].std()
print(f'Phish Mean: {phish_mean}, Phish Std: {phish_std}')

Phish Mean: 1.0, Phish Std: 0.0


In [58]:
# Bear case is negative two sigma from mean
phish_bear_pct = phish_mean - (2 * phish_std)
print(f'Phish Bear: {phish_bear_pct}')

# Base case is mean
phish_base_pct = phish_mean
print(f'Phish Base: {phish_base_pct}')

# Bull case is positive two sigma from mean (floor at 100%)
phish_bull_pct = min(phish_mean + (2 * phish_std), 1)
print(f'Phish Bull: {phish_bull_pct}')

for index, row in revenue_df.iterrows():
    if row['Group Name'] == 'Phish':
        revenue_df.at[index, 'Projected Attendance (Bear)'] = phish_bear_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bear)'] = phish_bear_pct
        revenue_df.at[index, 'Projected Attendance (Base)'] = phish_base_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Base)'] = phish_base_pct
        revenue_df.at[index, 'Projected Attendance (Bull)'] = phish_bull_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bull)'] = phish_bull_pct

revenue_df.head()

Phish Bear: 1.0
Phish Base: 1.0
Phish Bull: 1.0


,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,15906.168282,0.795308,18046.666667,0.902333,20000,1,0,0,0,0,0,0,0,0,0


# Dead & Co

In [59]:
# mean and std attendance for Dead & Co available in hist_stats
dead_mean = hist_stats['Dead&Co Attendance'].mean()
dead_std = hist_stats['Dead&Co Attendance'].std()
print(f'Dead&Co Mean: {dead_mean}, Dead&Co Std: {dead_std}')

Dead&Co Mean: 0.9323333333333332, Dead&Co Std: 0.029471737874331983


In [62]:
# Bear case is negative two sigma from mean
dead_bear_pct = dead_mean - (2 * dead_std)
print(f'Dead&Co Bear: {dead_bear_pct}')

# Base case is mean
dead_base_pct = dead_mean
print(f'Dead&Co Base: {dead_base_pct}')

# Bull case is positive two sigma from mean (floor at 100%)
dead_bull_pct = min(dead_mean + (2 * dead_std), 1)
print(f'Dead&Co Bull: {dead_bull_pct}')

for index, row in revenue_df.iterrows():
    if row['Group Name'] == 'Dead&Co':
        revenue_df.at[index, 'Projected Attendance (Bear)'] = dead_bear_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bear)'] = dead_bear_pct
        revenue_df.at[index, 'Projected Attendance (Base)'] = dead_base_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Base)'] = dead_base_pct
        revenue_df.at[index, 'Projected Attendance (Bull)'] = dead_bull_pct * row['Total Available Tickets']
        revenue_df.at[index, 'Projected Attendance % (Bull)'] = dead_bull_pct

revenue_df.head()

Dead&Co Bear: 0.8733898575846692
Dead&Co Base: 0.9323333333333332
Dead&Co Bull: 0.9912768090819972


,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,0,0,0,0,0,0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,0,0,0,0,0,0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,0,0,0,0,0,0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,0,0,0,0,0,0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,0,0,0,0,0,0,0,0,0


# Build Ticket Revenues

In [63]:
# Assume flat TR
TAKE_RATE = 0.11

u2_asp_mean = hist_stats['U2 ASP'].mean()
u2_asp_std = hist_stats['U2 ASP'].std()
phish_asp_mean = hist_stats['Phish ASP'].mean()
phish_asp_std = hist_stats['Phish ASP'].std()
dead_asp_mean = hist_stats['Dead&Co ASP'].mean()
dead_asp_std = hist_stats['Dead&Co ASP'].std()

for index, row in revenue_df.iterrows():
    if row['Group Name'] == 'U2 Residence':
        revenue_df.at[index, 'Projected Ticket Revenue (Bear)'] = row['Projected Attendance (Bear)'] * (u2_asp_mean - (u2_asp_std)) * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Base)'] = row['Projected Attendance (Base)'] * u2_asp_mean * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Bull)'] = row['Projected Attendance (Bull)'] * (u2_asp_mean + (u2_asp_std)) * TAKE_RATE
    if row['Group Name'] == 'Phish':
        revenue_df.at[index, 'Projected Ticket Revenue (Bear)'] = row['Projected Attendance (Bear)'] * (phish_asp_mean - (phish_asp_std)) * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Base)'] = row['Projected Attendance (Base)'] * phish_asp_mean * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Bull)'] = row['Projected Attendance (Bull)'] * (phish_asp_mean + (phish_asp_std)) * TAKE_RATE
    if row['Group Name'] == 'Dead&Co':
        revenue_df.at[index, 'Projected Ticket Revenue (Bear)'] = row['Projected Attendance (Bear)'] * (dead_asp_mean - (dead_asp_std)) * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Base)'] = row['Projected Attendance (Base)'] * dead_asp_mean * TAKE_RATE
        revenue_df.at[index, 'Projected Ticket Revenue (Bull)'] = row['Projected Attendance (Bull)'] * (dead_asp_mean + (dead_asp_std)) * TAKE_RATE

revenue_df.head()

,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,0,0,0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,0,0,0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,0,0,0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,0,0,0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,0,0,0,0,0,0


# F&B Sales

We do not have historicals on food and bev, so we have to make a few assumptions here while still assuming a reasonable mean and standard deviation estimate. For F&B, we will assume that U2, Phish, and Dead&Co fans are equally thirsty / hungry.

In [64]:
# Data taken from Sphere Menu
fnb_asps = {
    'Alcoholic Beverage ASP': 23.5,
    'Non-Alcoholic Beverage ASP': 8.75,
    'Food ASP': 21
}

# ASSUMPTION
fnb_consumption_rates = {
    'Alcohol Purchase Percent Mean': 0.5,
    'Alcohol Purchase Percent Std': 0.25,
    'Non-Alcohol Purchase Percent Mean': 0.6,
    'Non-Alcohol Purchase Percent Std': 0.3,
    'Food Purchase Percent Mean': 0.5,
    'Food Purchase Percent Std': 0.1
}

In [65]:
# apply to revenue_df
for index, row in revenue_df.iterrows():
    base_attendance = row['Projected Attendance (Base)']
    # bear case is sigma below mean
    bear_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * (fnb_consumption_rates['Alcohol Purchase Percent Mean'] - fnb_consumption_rates['Alcohol Purchase Percent Std'])),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * (fnb_consumption_rates['Non-Alcohol Purchase Percent Mean'] - fnb_consumption_rates['Non-Alcohol Purchase Percent Std'])),
        'Food': base_attendance * (fnb_asps['Food ASP'] * (fnb_consumption_rates['Food Purchase Percent Mean'] - fnb_consumption_rates['Food Purchase Percent Std']))
    }
    # base case is mean
    base_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * fnb_consumption_rates['Alcohol Purchase Percent Mean']),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * fnb_consumption_rates['Non-Alcohol Purchase Percent Mean']),
        'Food': base_attendance * (fnb_asps['Food ASP'] * fnb_consumption_rates['Food Purchase Percent Mean'])
    }
    # bull case is sigma above mean
    bull_fnb = {
        'Alcohol': base_attendance * (fnb_asps['Alcoholic Beverage ASP'] * (fnb_consumption_rates['Alcohol Purchase Percent Mean'] + fnb_consumption_rates['Alcohol Purchase Percent Std'])),
        'Non-Alcohol': base_attendance * (fnb_asps['Non-Alcoholic Beverage ASP'] * (fnb_consumption_rates['Non-Alcohol Purchase Percent Mean'] + fnb_consumption_rates['Non-Alcohol Purchase Percent Std'])),
        'Food': base_attendance * (fnb_asps['Food ASP'] * (fnb_consumption_rates['Food Purchase Percent Mean'] + fnb_consumption_rates['Food Purchase Percent Std']))
    }
    revenue_df.at[index, 'Projected F&B Revenue (Bear)'] = bear_fnb['Alcohol'] + bear_fnb['Non-Alcohol'] + bear_fnb['Food']
    revenue_df.at[index, 'Projected F&B Revenue (Base)'] = base_fnb['Alcohol'] + base_fnb['Non-Alcohol'] + base_fnb['Food']
    revenue_df.at[index, 'Projected F&B Revenue (Bull)'] = bull_fnb['Alcohol'] + bull_fnb['Non-Alcohol'] + bull_fnb['Food']

revenue_df.head()

,Group Name,Event Name,Date,Total Available Tickets,Projected Attendance (Bear),Projected Attendance % (Bear),Projected Attendance (Base),Projected Attendance % (Base),Projected Attendance (Bull),Projected Attendance % (Bull),Projected Ticket Revenue (Bear),Projected Ticket Revenue (Base),Projected Ticket Revenue (Bull),Projected F&B Revenue (Bear),Projected F&B Revenue (Base),Projected F&B Revenue (Bull),Projected Merch Revenue (Bear),Projected Merch Revenue (Base),Projected Merch Revenue (Bull)
0,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-28,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,304988.666667,496283.333333,687578.0,0,0,0
1,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-18,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,304988.666667,496283.333333,687578.0,0,0,0
2,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-01-27,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,304988.666667,496283.333333,687578.0,0,0,0
3,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-02-04,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,304988.666667,496283.333333,687578.0,0,0,0
4,U2 Residence,U2:UV Achtung Baby Live At Sphere - General Ad...,2024-03-03,20000,15906.168282,0.795308,18046.666667,0.902333,20000.0,1.0,539694.010588,662452.293727,789712.345846,304988.666667,496283.333333,687578.0,0,0,0


# Merch

# DViz

In [66]:
revenue_df.to_excel('data/sphr_concert_revenue_data.xlsx')